# Nelder - Mead with Simulated Annealing

## Function and bounds

In [26]:
import copy
import numpy as np
import math
from IPython.core.debugger import set_trace
# simulated annealing search of a one-dimensional objective function
from numpy import asarray
from numpy import exp
from numpy.random import randn
from numpy.random import rand
from numpy.random import seed
from matplotlib import pyplot


lowerBound, upperBound = 0, math.pi
x = np.random.uniform(low=lowerBound, high=upperBound, size=(10,))
N = len(x)
l = [lowerBound for i in range(10)]
u = [upperBound for i in range(10)]


def f(x):
    N = len(x)
    #Sphere, x_i in [-30,30]
    #Sum = 0
    #for i in range(0,N):
    #    Sum = Sum + (x[i])**2

    #Rosenbrock, x_i in [-30, -30]
    #Sum = 0
    #for i in range(0, N-1):
    #    Sum = Sum + 100 * ((x[i])**2 - x[i+1])**2 + (x[i] - 1)**2
        
    #Ackley, x_i in [-5, -10]
    #Sum1, Sum2 = 0, 0
    #for i in range(0,N):
    #    Sum1 = Sum1 + (x[i])**2
    #    Sum2 = Sum2 + math.cos(2*math.pi*x[i])
    #Sum= -20.0*math.exp(-0.2*math.sqrt((1/N)*(Sum1)))-math.exp((1/N) * Sum2) + math.e + 20
    
    #Griewank, x_i in [-600,600]
    d = 4000; Sum = 0
    for i in range(0,N): Sum = Sum + (x[i]-100)**2
    Sum = Sum/d; Prod = 1
    for i in range(0,N): Prod = Prod * math.cos((x[i]-100)/math.sqrt(i+1))
    Sum = Sum - Prod +1
    
    #Michalewicz, x_i in [0,math.pi]
    #Sum = 0; m = 10
    #for i in range(0,N): Sum = Sum - math.sin(x[i])*(math.sin(((i+1)*(x[i])**2)/math.pi))**(2*m)
    
    #return value, don't delete
    return Sum

# Algorithm

In [27]:
def _e(index,size):
    arr = np.zeros(size)
    arr[index] = 1.0
    return arr



def Delta(x):
    Sum = 0
    #no general, only box constraints
    for i in range(0,N):
        Sum = Sum + max(x[i]-u[i],0) + max(l[i]-x[i], 0)
    return Sum



def better(x,y):
    Dx = Delta(x)
    Dy = Delta(y)
    if(Dx < Dy):
        return True
    elif(Dx == Dy and f(x) < f(y)):
        return True
    else:
        return False
    
    
    
def findExtremes(s):
    Ds = np.empty([N+1])
    fs = np.empty([N+1])
    
    for i in range(0, N+1):
        Ds[i] = Delta(s[i])
        fs[i] = f(s[i])
    
    arg_b = np.argmin(Ds)
    for i in range(0, N+1):
        if Ds[i] == Ds[arg_b] and i != arg_b:
            if(fs[i] < fs[arg_b]):
                arg_b = i;
                
    arg_w = np.argmax(Ds)
    for i in range(0, N+1):
        if Ds[i] == Ds[arg_w] and i != arg_w:
            if(fs[i] > fs[arg_w]):
                arg_w = i;
    
    #mask the worst to find the second worst
    Ds[arg_w] = 0
    
    arg_sw = np.argmax(Ds)
    for i in range(0, N+1):
        if Ds[i] == Ds[arg_sw] and i != arg_w:
            if(fs[i] > fs[arg_sw]):
                arg_sw = i;
    
    return arg_b,arg_w,arg_sw



def centroid(s,argWorst):
    return np.mean(np.delete(s, argWorst), axis = 0)


def SimplexLocalSearch(x,Epsilon,Lambda,Alpha=1,Gamma=2,Beta=-0.5):
    N = len(x)
    s = np.empty([N+1,N])
    s[0] = x
    for j in range(1,N+1):
        s[j] = x + Lambda*(u[j-1]-l[j-1])*_e(j-1,N);
    k = 0
    
    
    argBest, argWorst, argSecondWorst = findExtremes(s)
    sBest, sWorst, sSecondWorst = s[argBest], s[argWorst], s[argSecondWorst]
    #set_trace()
    while abs(f(sBest) - f(sWorst)) > Epsilon and k < 1000:
        #set_trace()
        #if(k%1000 == 0):print(f(sBest),'\n')
        r = (1+Alpha)*centroid(s,argWorst) - Alpha*sWorst
        if(better(sBest,r) and better(r,sSecondWorst)):
            s[argWorst] = r
        elif(better(r,sBest)):
            e = (1+Gamma)*centroid(s,argWorst)-Gamma*sWorst
            if(better(e,r)):
                s[argWorst] = e
            else:
                s[argWorst] = r
        else:
            c = (1+Beta)*centroid(s,argWorst)-Beta*sWorst
            if(better(c,sWorst)):
                s[argWorst] = c
            else:
                for j in range(0,N+1):
                    s[j] = (s[j] + sBest)/2

        #set_trace()
        argBest, argWorst, argSecondWorst = findExtremes(s)
        sBest, sWorst, sSecondWorst = s[argBest], s[argWorst], s[argSecondWorst]
        k = k + 1
    #print(f(sBest))
    return sBest



def SimulatedAnnealing(n_iterations, Epsilon, Lambda, Temperature):
    # generate an initial point
    best = np.random.uniform(low=lowerBound, high=upperBound, size=(10,))
    # evaluate the initial point
    best_eval = f(best)
    # current working solution
    curr, curr_eval = best, best_eval
    scores = list()
    # run the algorithm
    for i in range(n_iterations):
        # take a step
        candidate = SimplexLocalSearch(best,Epsilon,Lambda)
        #set_trace()
        # evaluate candidate point
        candidate_eval = f(candidate)
        # check for new best solution
        if candidate_eval < best_eval:
            # store new best point
            best, best_eval = candidate, candidate_eval
            # keep track of scores
            scores.append(best_eval)
            # report progress
            print('>%d f(%s) = %s' % (i, best, best_eval))
        # difference between candidate and current point evaluation
        diff = candidate_eval - curr_eval
        # calculate temperature for current epoch
        t = Temperature / float(i + 1)
        # calculate metropolis acceptance criterion
        metropolis = exp(-diff / t)
        # check if we should keep the new point
        if diff < 0 or rand() < metropolis:
            # store the new current point
            curr, curr_eval = candidate, candidate_eval
    return [best, best_eval, scores]

## Main

In [ ]:
Lambda = 10
n_iterations = 20
Epsilon = 0
Temperature = 100
# perform the simulated annealing search
best, score, scores = SimulatedAnnealing(n_iterations,Epsilon,Lambda,Temperature)
print('Done!')
print('f(%s) = %s' % (best, score))
# line plot of best scores
pyplot.plot(scores, '.-')
pyplot.xlabel('Improvement Number')
pyplot.ylabel('Evaluation f(x)')
pyplot.show()

>0 f([3.04829295 3.05397562 3.05494306 3.05748989 3.05125464 3.05438514
 3.04965347 3.05869299 3.05563258 2.92751717]) = 24.494790952456345
>1 f([3.14037389 3.14037372 3.14037369 3.14037361 3.1403738  3.1403737
 3.13989448 3.14037357 3.14037367 3.13989821]) = 24.448203347920703
>2 f([3.14155075 3.14155075 3.14155075 3.14155075 3.14155075 3.14155075
 3.14155075 3.14155075 3.14158071 3.14155075]) = 24.447605333275234
>3 f([3.14159234 3.14159234 3.14159234 3.14159234 3.14159234 3.1415914
 3.14159234 3.14159234 3.14159234 3.14159234]) = 24.447587345270918
>4 f([3.14159259 3.14159259 3.14159259 3.14159259 3.14159259 3.14159259
 3.14159259 3.14159259 3.14159259 3.14159247]) = 24.447587196389453
>5 f([3.14159265 3.14159265 3.14159265 3.14159265 3.14159265 3.14159265
 3.14159265 3.14159265 3.14159265 3.14159265]) = 24.44758716027387
>6 f([3.14159265 3.14159265 3.14159265 3.14159265 3.14159265 3.14159265
 3.14159265 3.14159265 3.14159265 3.14159265]) = 24.447587159523483
